In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *
from datetime import datetime

In [2]:
config = SparkConf().setAll([
    ('spark.executor.cores', '2'), ('spark.executor.memory', '4g'), ('spark.driver.memory','4g'), ('spark.submit.deployMode','client')
])
sc.stop()
sc = SparkContext(conf=config)

#sc = SparkContext()
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
sc.getConf()#.get("spark.executor.instances")

In [3]:
#df = sqlContext.read.csv('gs://kaggle01_test/notebooks/jupyter/pyspark-ecommerce-prediction-main/ecommerce-data100.csv', header=True)
#df = sqlContext.read.csv('ecommerce-data.csv', header=True)
df = sqlContext.read.csv('user_ecommerce-data.csv', header=True, inferSchema=True)
df = df.drop('CustomerID', 'UnitPrice')
df = df.withColumn("Date", to_date(col("Date")))
#df = df.withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), 'mm-dd-yyyy HH:mm:ss'))

#df.show()
#df = df.drop_duplicates(['CustomerID'])
#df = df.na.drop()
# df.dropna(subset=['Description']).show()
df.count()

397884

In [4]:
def get_outliers(df, column):
    quantiles = df.stat.approxQuantile(column, [0.05, 0.95], 0.0)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    IQR = Q3 - Q1
    lowerRange = Q1 - 1.5 * IQR
    upperRange = Q3 + 1.5 * IQR
    df.filter((df[column] < lowerRange) | (df[column] > upperRange)).show()

def remove_outliers(df, column):
    quantiles = df.stat.approxQuantile(column, [0.05, 0.95], 0.0)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    IQR = Q3 - Q1
    lowerRange = Q1 - 1.5 * IQR
    upperRange = Q3 + 1.5 * IQR
    df = df.filter((df[column] >= lowerRange) | (df[column] <= upperRange))

remove_outliers(df, 'Quantity')
remove_outliers(df, 'TotalPrice')

In [5]:
agg = df.select('Date', 'Quantity', 'TotalPrice').groupby('Date').sum()
agg = agg.withColumnRenamed('sum(Quantity)', 'Quantity')
agg = agg.withColumnRenamed('sum(TotalPrice)', 'TotalPrice')

agg = agg.withColumn('Month', month('Date'))
agg = agg.withColumn('DayOfMonth', dayofmonth('Date'))
agg = agg.withColumn('DayOfWeek', dayofweek('Date'))
agg = agg.withColumn('DayOfYear', dayofyear('Date'))
agg = agg.withColumn('Weekend', dayofweek('Date').isin(6,7).cast('int'))

#agg.show()
#agg.write.csv('forecasting-agg.csv', header=True)

In [6]:
agg.count()

305

In [7]:
from pyspark.sql.functions import desc, asc
agg = agg.orderBy(asc("Date"))

In [8]:
pd_agg = agg.toPandas()

In [9]:
pd_agg['Date'].count() * 0.9, pd_agg['Date'].count() * 0.1

(274.5, 30.5)

In [10]:
275+30

305

In [11]:
from pyspark.sql.functions import desc, asc
agg = agg.orderBy(asc("Date"))
agg_train = agg.limit(275)
agg_test = agg.orderBy(desc("Date")).limit(30).orderBy("Date")
#train.show()
#test.show()

In [175]:
pd_train = agg_train.toPandas()
pd_test = agg_test.toPandas()
pd_train['Date'].unique(), pd_test['Date'].unique()

(array([datetime.date(2010, 12, 1), datetime.date(2010, 12, 2),
        datetime.date(2010, 12, 3), datetime.date(2010, 12, 5),
        datetime.date(2010, 12, 6), datetime.date(2010, 12, 7),
        datetime.date(2010, 12, 8), datetime.date(2010, 12, 9),
        datetime.date(2010, 12, 10), datetime.date(2010, 12, 12),
        datetime.date(2010, 12, 13), datetime.date(2010, 12, 14),
        datetime.date(2010, 12, 15), datetime.date(2010, 12, 16),
        datetime.date(2010, 12, 17), datetime.date(2010, 12, 19),
        datetime.date(2010, 12, 20), datetime.date(2010, 12, 21),
        datetime.date(2010, 12, 22), datetime.date(2010, 12, 23),
        datetime.date(2011, 1, 4), datetime.date(2011, 1, 5),
        datetime.date(2011, 1, 6), datetime.date(2011, 1, 7),
        datetime.date(2011, 1, 9), datetime.date(2011, 1, 10),
        datetime.date(2011, 1, 11), datetime.date(2011, 1, 12),
        datetime.date(2011, 1, 13), datetime.date(2011, 1, 14),
        datetime.date(2011, 1, 16

In [ ]:
from pyspark.ml.feature import Bucketizer

bucketizer = Bucketizer(splits=[ 0, 2, 5, 8, 11, 14, 15, 5000], inputCol="Quantity", outputCol="QuantityRange")
df = bucketizer.setHandleInvalid("keep").transform(df)

bucketizer = Bucketizer(splits=[ 0, 1, 2, 3, 4, 20], inputCol="UnitPrice", outputCol="PriceRange")
df = bucketizer.setHandleInvalid("keep").transform(df)

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="Month", outputCol="DateRange")

result = discretizer.fit(df).transform(df)

In [ ]:
from pyspark.ml.feature import StringIndexer

si = StringIndexer(inputCol='StockCode', outputCol='StockCodeIndex')
df = si.fit(df).transform(df)

si = StringIndexer(inputCol='Country', outputCol='CountryIndex')
df = si.fit(df).transform(df)

#df.show()

In [ ]:
df = df.withColumn("CountryIndex", df["CountryIndex"].cast('int'))
df = df.withColumn("StockCodeIndex", df["StockCodeIndex"].cast('int'))

df = df.withColumn("QuantityRange", df["QuantityRange"].cast('int'))
df = df.withColumn("PriceRange", df["PriceRange"].cast('int'))

df = df.withColumn("Cluster", df["Cluster"].cast('int'))

In [ ]:
#agg = sqlContext.read.csv('forecasting-agg.csv', header=True, inferSchema=True)

In [12]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    #inputCols=['Quantity', 'UnitPrice', 'QuantityRange', 'PriceRange', 'Month', 'CountryIndex', 'StockCodeIndex', 'Cluster'],
    inputCols=['Quantity','TotalPrice','Month','DayOfMonth','DayOfWeek','DayOfYear','Weekend'],
    #inputCols=['Quantity','TotalPrice','Month','DayOfMonth','DayOfWeek'],
    #inputCols=['Quantity','TotalPrice','Month','DayOfWeek'],
    #inputCols=['Quantity','TotalPrice','Month'],
    #inputCols=['Quantity','TotalPrice'],
    outputCol="features")

train = assembler.transform(agg_train).select(['features', 'TotalPrice'])
test = assembler.transform(agg_test).select(['features', 'TotalPrice'])

In [13]:
def evaluate(prediction):
    evaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="rmse")
    print(f"Root Mean Squared Error (RMSE) on test data = {evaluator.evaluate(prediction)}")

    evaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="mse")
    print(f"Mean Squared Error (MSE) on test data = {evaluator.evaluate(prediction)}")

    evaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="mae")
    print(f"MAE on test data = {evaluator.evaluate(prediction)}")

    evaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="r2")
    print(f"R2 on test data = {evaluator.evaluate(prediction)}")


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression

#lr = LinearRegression(maxIter=5, regParam=0.3, elasticNetParam=0.8)
lr = LinearRegression(labelCol='TotalPrice')
lrparamGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.001, 0.01, 0.1, 0.5, 1.0, 2.0])
             #.addGrid(lr.regParam, [0.01, 0.1, 0.5])
             .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0])
             #.addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10, 20, 50])
             #.addGrid(lr.maxIter, [1, 5, 10])
             .build())
lrevaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="rmse")

lrcv = CrossValidator(estimator=lr,
                          estimatorParamMaps=lrparamGrid,
                          evaluator=lrevaluator,
                          numFolds=5)

lrcvModel = lrcv.fit(train)

#lrcvSummary = lrcvModel.bestModel.summary
#print("Coefficient Standard Errors: " + str(lrcvSummary.coefficientStandardErrors))
#print("P Values: " + str(lrcvSummary.pValues)) # Last element is the intercept

lrpredictions = lrcvModel.transform(test)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluate(lrpredictions)

Root Mean Squared Error (RMSE) on test data = 0.0029825441442871987
Mean Squared Error (MSE) on test data = 8.895569572621857e-06
MAE on test data = 0.0015331794440664288
R2 on test data = 0.9999999999999852


In [20]:
import numpy as np

In [22]:
lrcvModel.getEstimatorParamMaps()[ np.argmax(lrcvModel.avgMetrics) ]

{Param(parent='LinearRegression_0a7febd1d8cb', name='regParam', doc='regularization parameter (>= 0).'): 0.5,
 Param(parent='LinearRegression_0a7febd1d8cb', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LinearRegression_0a7febd1d8cb', name='maxIter', doc='max number of iterations (>= 0).'): 1}

In [15]:
evaluate(lrpredictions)

Root Mean Squared Error (RMSE) on test data = 0.029824941532026635
Mean Squared Error (MSE) on test data = 0.0008895271373888073
MAE on test data = 0.015333228396560695
R2 on test data = 0.9999999999985182


In [78]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol='TotalPrice')
dtparamGrid = (ParamGridBuilder()
             #.addGrid(dt.maxDepth, [2, 5, 10, 20, 30])
             .addGrid(dt.maxDepth, [2, 5, 10])
             #.addGrid(dt.maxBins, [10, 20, 40, 80, 100])
             .addGrid(dt.maxBins, [10, 20])
             .build())
dtevaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="rmse")

dtcv = CrossValidator(estimator=dt,
                          estimatorParamMaps=dtparamGrid,
                          evaluator=dtevaluator,
                          numFolds=3)

dtcvModel = dtcv.fit(train)
dtpredictions = dtcvModel.transform(test)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluate(dtpredictions)

Root Mean Squared Error (RMSE) on test data = 15660.475405175966
Mean Squared Error (MSE) on test data = 245250489.91612136
MAE on test data = 3000.313295131763
R2 on test data = 0.5914366154999713


In [81]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(labelCol='TotalPrice')
gbtparamGrid = (ParamGridBuilder()
             #.addGrid(gbt.maxDepth, [2, 5, 10, 20, 30])
             #.addGrid(gbt.maxDepth, [2, 5, 10])
                .addGrid(gbt.maxDepth, [2])
             #.addGrid(gbt.maxBins, [10, 20, 40, 80, 100])
             #.addGrid(gbt.maxBins, [10, 20])
                .addGrid(gbt.maxBins, [10])
             .build())
gbtevaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="r2")

gbtcv = CrossValidator(estimator=gbt,
                          estimatorParamMaps=gbtparamGrid,
                          evaluator=gbtevaluator,
                          numFolds=3)

gbtcvModel = gbtcv.fit(train)
gbtpredictions = gbtcvModel.transform(test)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluate(gbtpredictions)

Root Mean Squared Error (RMSE) on test data = 16503.784329678052
Mean Squared Error (MSE) on test data = 272374897.20052683
MAE on test data = 4397.349797944288
R2 on test data = 0.5462499997812253


In [82]:
best_model = gbtcvModel.bestModel

In [94]:
best_gbtpredictions = best_model.transform(train)#gbtcvModel.avgMetrics

In [95]:
evaluate(best_gbtpredictions)

Root Mean Squared Error (RMSE) on test data = 4025.994853766512
Mean Squared Error (MSE) on test data = 16208634.562554438
MAE on test data = 2415.1828875461715
R2 on test data = 0.936428018516429


In [ ]:
best_model.transform(test)

In [80]:
evaluate(gbtpredictions)

Root Mean Squared Error (RMSE) on test data = 15523.706758546516
Mean Squared Error (MSE) on test data = 240985471.52534276
MAE on test data = 2939.6750230249017
R2 on test data = 0.5985417199557761


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(labelCol='TotalPrice')
rfparamGrid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
             .addGrid(rf.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
             .addGrid(rf.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
             .addGrid(rf.numTrees, [5, 20, 50])
             .build())
rfevaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="rmse")

rfcv = CrossValidator(estimator=rf,
                          estimatorParamMaps=rfparamGrid,
                          evaluator=rfevaluator,
                          numFolds=3,
                          parallelism=10
                     )
start = datetime.now()
rfcvModel = rfcv.fit(train)
print(f'fitting: {datetime.now() - start}')
start = datetime.now()
rfpredictions = rfcvModel.transform(test)
print(f'predicting: {datetime.now() - start}')

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluate(rfpredictions)

rfparamGrid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
             .addGrid(rf.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
             .addGrid(rf.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
             .addGrid(rf.numTrees, [5, 20, 50])
             .build())
rfevaluator = RegressionEvaluator(labelCol="TotalPrice", predictionCol="prediction", metricName="rmse")

rfcv = CrossValidator(estimator=rf,
                          estimatorParamMaps=rfparamGrid,
                          evaluator=rfevaluator,
                          numFolds=5)

Root Mean Squared Error (RMSE) on test data = 14398.532319895226
Mean Squared Error (MSE) on test data = 207317732.96706742
MAE on test data = 2788.622927146411
R2 on test data = 0.6546288870743233

In [30]:
evaluate(rfpredictions)

Root Mean Squared Error (RMSE) on test data = 14398.532319895226
Mean Squared Error (MSE) on test data = 207317732.96706742
MAE on test data = 2788.622927146411
R2 on test data = 0.6546288870743233


In [211]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

for md in [30]:
    for mb in [200]:
        print(f'maxDepth={md},maxBins={mb}')
        gbt = GBTRegressor(labelCol='TotalPrice',maxDepth=md,maxBins=mb)
        start = datetime.now()
        gbtModel = gbt.fit(train)
        print(f'fitting: {datetime.now() - start}')
        start = datetime.now()
        prediction = gbtModel.transform(test)
        print(f'predicting: {datetime.now() - start}')

        #print(gbtModel.featureImportances)
        #evaluate(gbtModel.transform(train))
        evaluate(gbtModel.transform(test))
#maxDepth=30,maxBins=50 0.4

maxDepth=30,maxBins=200
fitting: 0:00:25.708209
predicting: 0:00:00.022701
Root Mean Squared Error (RMSE) on test data = 14798.19619096638
Mean Squared Error (MSE) on test data = 218986610.5063319
MAE on test data = 3215.767711410018
R2 on test data = 0.7293016980126208


In [220]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

for md in [30]:
    for mb in [540]:
        for nt in [25]:
            print(f'maxDepth={md},maxBins={mb},numTrees={nt}')
            rf = RandomForestRegressor(labelCol='TotalPrice',maxDepth=md,maxBins=mb,numTrees=nt)
            start = datetime.now()
            rfModel = rf.fit(train)
            print(f'fitting: {datetime.now() - start}')
            start = datetime.now()
            prediction = rfModel.transform(test)
            print(f'predicting: {datetime.now() - start}')

            print(rfModel.featureImportances)
            #evaluate(rfModel.transform(train))
            evaluate(rfModel.transform(test))
            print('')

maxDepth=30,maxBins=540,numTrees=25
fitting: 0:00:06.783697
predicting: 0:00:00.031755
(7,[0,1,2,3,4,5,6],[0.23178971172965304,0.6769539587106376,0.020128899062635965,0.012058116906790867,0.04466457579549625,0.014242206035673768,0.00016253175911252798])
Root Mean Squared Error (RMSE) on test data = 18108.17150612983
Mean Squared Error (MSE) on test data = 327905875.2954123
MAE on test data = 4576.887264857051
R2 on test data = 0.5946621419048501



In [187]:
GBTRegressor

Root Mean Squared Error (RMSE) on test data = 19197.50159968777
Mean Squared Error (MSE) on test data = 368544067.67001444
MAE on test data = 7856.933766853909
R2 on test data = 0.31343925428569874


In [186]:
evaluate(rfModel.transform(train))

Root Mean Squared Error (RMSE) on test data = 2821.6695211504534
Mean Squared Error (MSE) on test data = 7961818.886589429
MAE on test data = 1035.8291532099954
R2 on test data = 0.9564494735427892


In [ ]:
print('ok')

In [14]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

for mi in [5,10]:
    for rp in [1]:
        for en in [0.5, 1]:
            print(f'maxIter={mi},regParam={rp},elasticNetParam={en}')
            lr = LinearRegression(labelCol='TotalPrice', maxIter=mi, regParam=rp, elasticNetParam=en)
            start = datetime.now()
            lrModel = lr.fit(train)
            print(f'fitting: {datetime.now() - start}')
            start = datetime.now()
            prediction = lrModel.transform(test)
            print(f'predicting: {datetime.now() - start}')

            #print(lrModel.featureImportances)
            #evaluate(lrModel.transform(train))
            evaluate(lrModel.transform(test))
            print('')

maxIter=5,regParam=1,elasticNetParam=0.5
fitting: 0:00:20.168227
predicting: 0:00:00.019501
Root Mean Squared Error (RMSE) on test data = 2534.4585315564923
Mean Squared Error (MSE) on test data = 6423480.048179491
MAE on test data = 1383.9785043579686
R2 on test data = 0.9920596737039239

maxIter=5,regParam=1,elasticNetParam=1
fitting: 0:00:08.235003
predicting: 0:00:00.017723
Root Mean Squared Error (RMSE) on test data = 2533.7734799090654
Mean Squared Error (MSE) on test data = 6420008.047490494
MAE on test data = 1383.7326368550755
R2 on test data = 0.99206396558592

maxIter=10,regParam=1,elasticNetParam=0.5
fitting: 0:00:06.798363
predicting: 0:00:00.014347
Root Mean Squared Error (RMSE) on test data = 558.3785398931979
Mean Squared Error (MSE) on test data = 311786.5938132597
MAE on test data = 379.82250520501606
R2 on test data = 0.999614587844743

maxIter=10,regParam=1,elasticNetParam=1
fitting: 0:00:06.592279
predicting: 0:00:00.014787
Root Mean Squared Error (RMSE) on test da

In [15]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='TotalPrice', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train)

print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

#trainingSummary.residuals.show()

prediction = lrModel.transform(test)
evaluate(prediction)

Coefficients: [0.04654540268024453,0.9826257680094383,3.0871702069784193,-0.5985103700892468,-244.39200825273446,0.09882774813417344,443.65606303735547]
Intercept: 501.6111374428941
numIterations: 11
objectiveHistory: [0.5000000000000002, 0.36030420263345214, 0.1135579263279735, 0.04392768755876902, 0.01911365596350906, 0.01605146499418765, 0.011711971940388342, 0.004945097905512382, 0.0041361614504214685, 0.0027977211384173175, 0.0002987365135778784]
RMSE: 344.765390
r2: 0.999441
Root Mean Squared Error (RMSE) on test data = 396.9319101074172
Mean Squared Error (MSE) on test data = 157554.94126152265
MAE on test data = 332.0016169723292
R2 on test data = 0.999805239895852


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
test.collect()

In [ ]:
a = np.array(agg.collect())

In [ ]:
periods = np.array(agg.collect())[:,6]

In [ ]:
len(a[:305,2])

In [ ]:
len(periods)

In [ ]:
t = np.array(agg.drop('Date').collect()) 

In [ ]:
t[:,1].mean()

In [ ]:
p = np.array(prediction.select('prediction').collect())

In [ ]:
pf = p[:].flatten()

In [ ]:
len(pf)

In [ ]:
plt.plot(np.sort(periods), a[:,2])
plt.plot(np.sort(periods), pf)

In [ ]:
a[:5,2]